# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import gc
import re
import nltk
import string
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

# Import and Combine Data

In [ ]:
df1 = pd.read_csv("Dataset/Fake.csv")
df2 = pd.read_csv("Dataset/True.csv")

df1['label'] = 1
df2['label'] = 0 

df1 = df1.drop(columns = ['subject', 'date'])
df2 = df2.drop(columns = ['subject', 'date'])

df3 = pd.read_csv("Dataset/news.csv")

# Use replace to change 'FAKE' to 1 and 'REAL' to 0
df3['label'] = df3['label'].replace({'FAKE': 1, 'REAL': 0})
df3 = df3.drop(columns = df3.columns[0])

df4 = pd.read_csv("Dataset/WELFake_Dataset.csv")

df4 = df4.drop(columns = df4.columns[0])
df4['label'] = df4['label'].replace({1: 0, 0: 1})

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
# Assuming 'df' is the combined DataFrame
true_count = df[df['label'] == 0].shape[0]
false_count = df[df['label'] == 1].shape[0]

# Create a pie chart
labels = ['True', 'False']
sizes = [true_count, false_count]
colors = ['lightblue', 'lightcoral']
explode = (0.1, 0)  # explode the 1st slice (True)

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution of True and False News')
plt.show()

# Data Processing

In [ ]:
#Remove Rows with empty values
print(df.shape[0])
df = df.dropna()
print(df.shape[0])

In [ ]:
#Remove duplciate data
print(df.shape[0])
df = df.drop_duplicates(subset=['title'], keep='first')
print(df.shape[0])

In [ ]:
# Shuffle (randomize) the rows
df = df.sample(frac=1, random_state=42) 

In [ ]:
#Title and label only
df = df[['title','label']]

In [ ]:
# Apply preprocessing: remove punctuation by character
df['title'] = df['title'] .apply(lambda x: x.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))))

#Remove Numbers
df['title']= df['title'].str.replace(r'\b\d+\b', '', regex=True)

# Load English stopwords
stopwords_set = set(stopwords.words('english'))

# Apply preprocessing: remove stopwords
df['title'] = df['title'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.lower() not in stopwords_set]))

# Apply preprocessing: convert text to lowercase
df['title'] = df['title'].apply(lambda x: x.lower())

#Reduce words to root form
stemmer = PorterStemmer()
df['title'] = df['title'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [ ]:
# Assuming 'df' is the DataFrame containing 'title' and 'label'
df['title_word_count'] = df['title'].apply(lambda x: len(str(x).split()))

# Plotting the histogram
plt.hist(df['title_word_count'], bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of Title Word Count')
plt.xlabel('Number of Words in Title')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Save DataFrame
df.to_pickle('Dataframe.pkl')

In [ ]:
# To load the DataFrame back
df = pd.read_pickle('../Dataframe.pkl')

## BERT Model (Transfer Learning)

### Call BERT Model and Tokeniser

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

### Tokenise

In [ ]:
#Bert tokeniser
max_length = 20

tokenized_data = tokenizer.batch_encode_plus(df['title'].tolist(), max_length=max_length, pad_to_max_length=True, truncation=True, return_tensors='pt')
labels = torch.tensor(df['label'].tolist())

### Data Split

In [ ]:
# Split the data into training and testing sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_data.input_ids,
    labels,
    test_size=0.2,
)

### DataLoader and Calling GPU

In [ ]:
#Convert to PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Move the model and data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)
test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

### Train Model

In [ ]:
# Training loop with tqdm for progress bar
epochs = 4

train_losses = []  # List to store training losses
train_accuracies = []  # List to store training accuracies
test_losses = []  # List to store test losses
test_accuracies = []  # List to store test accuracies
best_validation_loss = float('inf') 

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Create a tqdm progress bar for training
    progress_bar = tqdm(train_dataloader)
    total_loss = 0.0  # Variable to store the total training loss
    correct_predictions = 0  # Variable to store the number of correct predictions
    
    for batch_inputs, batch_labels in progress_bar:
        # Move data to GPU
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
        
        # Forward pass
        outputs = model(batch_inputs)
        logits = outputs.logits
        
        # Calculate loss
        loss = criterion(logits, batch_labels)
        total_loss += loss.item() * len(batch_labels)  # Accumulate the loss
        
        # Calculate number of correct predictions
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(predictions == batch_labels).item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update progress bar
        progress_bar.set_postfix({'Loss': loss.item()})
    
    # Calculate average training loss and accuracy for the epoch
    average_train_loss = total_loss / len(train_dataloader.dataset)
    train_accuracy = correct_predictions / len(train_dataloader.dataset)
    print(f"Training Loss: {average_train_loss}")
    print(f"Training Accuracy: {train_accuracy}")
    
    train_losses.append(average_train_loss)
    train_accuracies.append(train_accuracy)
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs)
        test_logits = test_outputs.logits
        test_loss = criterion(test_logits, test_labels).item()
        test_losses.append(test_loss)
        
        # Calculate test accuracy
        test_predictions = torch.argmax(test_logits, dim=1)
        test_accuracy = accuracy_score(test_labels.cpu().numpy(), test_predictions.cpu().numpy())
        test_accuracies.append(test_accuracy)
    
    print(f"Test Loss after Epoch {epoch + 1}: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    
    # Check if the current model has the best validation loss
    if test_loss < best_validation_loss:
        best_validation_loss = test_loss
        # Save the current best model
        torch.save(model.state_dict(), 'best_model.pth')

In [ ]:
# Load the best model
best_model = model
best_model.load_state_dict(torch.load('best_model.pth'))

### Evaluating Model

#### Basic Evaluaiton

In [ ]:
# Set the model to evaluation mode
best_model.eval()

# Convert NumPy arrays to PyTorch tensors
test_inputs_tensor = torch.tensor(test_inputs).to(device)
test_labels_tensor = torch.tensor(test_labels).to(device)

# Forward pass
with torch.no_grad():
    test_outputs = best_model(test_inputs_tensor)
    test_logits = test_outputs.logits
    test_predictions = torch.argmax(test_logits, dim=1)

# Convert predictions and labels to numpy arrays
test_predictions = test_predictions.cpu().numpy()
test_labels = test_labels_tensor.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

# Print the metrics
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")

#### Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Normalize the confusion matrix
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Set the style
sns.set(style="whitegrid")

# Plot the normalized confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cbar=True,
            xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])

plt.title('Base BERT Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

#### Receiver Operating Characteristic (ROC) Curve and Area Under Curve (AUC)

In [ ]:
test_logits_cpu = test_logits.cpu().numpy()

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(test_labels, test_logits_cpu[:, 1])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Base BERT - ROC')
plt.legend(loc="lower right")
plt.show()

## BERT Model (Transfer Learning with layer freezing and custom final layers)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = AutoModel.from_pretrained('bert-base-uncased')

### Freeze Layers and Add Custom Layers

In [ ]:
# Freeze BERT layers
for param in bert.parameters():
    param.requires_grad = False
    
# Add final layers
class CustomBertModel(nn.Module):
    def __init__(self, bert, num_classes=2):
        super(CustomBertModel, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        
        # Extract [CLS] token from the last hidden state
        pooled_output = outputs.last_hidden_state[:, 0, :]

        x = self.dropout(pooled_output)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

    
# Assuming 'device' is already defined
model = CustomBertModel(bert)

### Tokenise 

In [ ]:
#Bert tokeniser
max_length = 20

tokenized_data = tokenizer.batch_encode_plus(df['title'].tolist(), max_length=max_length, pad_to_max_length=True, truncation=True, return_tensors='pt')
labels = torch.tensor(df['label'].tolist())

In [ ]:
# Save tokenized data
torch.save(tokenized_data, 'tokenized_data.pt')

# Save labels
torch.save(labels, 'labels.pt')

### Data Split

In [ ]:
# Split the data into training and testing sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_data.input_ids,
    labels,
    test_size=0.2,
)

### DataLoader and Calling GPU

In [ ]:
#Convert to PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Move the model and data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)
test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

### Train Model

In [ ]:
# Training loop with tqdm for progress bar
epochs = 4

train_losses = []  # List to store training losses
train_accuracies = []  # List to store training accuracies
test_losses = []  # List to store test losses
test_accuracies = []  # List to store test accuracies
best_validation_loss = float('inf')  # Initialize with a very large value

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Create a tqdm progress bar for training
    progress_bar = tqdm(train_dataloader)
    total_loss = 0.0  # Variable to store the total training loss
    correct_predictions = 0  # Variable to store the number of correct predictions

    model.train()  # Set the model to training mode

    for batch_inputs, batch_labels in progress_bar:
        # Move data to GPU
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_inputs, attention_mask=(batch_inputs != 0).float())
        logits = outputs

        # Calculate loss
        loss = criterion(logits, batch_labels)
        total_loss += loss.item() * len(batch_labels)  # Accumulate the loss

        # Calculate number of correct predictions
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(predictions == batch_labels).item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        progress_bar.set_postfix({'Loss': loss.item()})

    # Calculate average training loss and accuracy for the epoch
    average_train_loss = total_loss / len(train_dataloader.dataset)
    train_accuracy = correct_predictions / len(train_dataloader.dataset)
    print(f"Training Loss: {average_train_loss}")
    print(f"Training Accuracy: {train_accuracy}")

    train_losses.append(average_train_loss)
    train_accuracies.append(train_accuracy)

    # Evaluate the model on the test set after each epoch
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs, attention_mask=(test_inputs != 0).float())
        test_logits = test_outputs
        test_loss = criterion(test_logits, test_labels).item()
        test_losses.append(test_loss)

        # Calculate test accuracy
        test_predictions = torch.argmax(test_logits, dim=1)
        test_accuracy = accuracy_score(test_labels.cpu().numpy(), test_predictions.cpu().numpy())
        test_accuracies.append(test_accuracy)

    print(f"Test Loss after Epoch {epoch + 1}: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

    # Check if the current model has the best validation loss
    if test_loss < best_validation_loss:
        best_validation_loss = test_loss
        # Save the current best model
        torch.save(model.state_dict(), 'best_model_1.pth')

In [ ]:
# Load the best model
best_model_1 = model
best_model_1.load_state_dict(torch.load('best_model_1.pth'))

### Evaluating the model

#### Basic Evaluation

In [ ]:
# Set the model to evaluation mode
best_model_1.eval()

# Convert NumPy arrays to PyTorch tensors
test_inputs_tensor = torch.tensor(test_inputs).to(device)
test_attention_mask_tensor = (test_inputs_tensor != 0).float().to(device)
test_labels_tensor = torch.tensor(test_labels).to(device)

# Forward pass
with torch.no_grad():
    test_outputs = best_model_1(test_inputs_tensor, attention_mask=test_attention_mask_tensor)
    test_logits = test_outputs

# Perform necessary operations on logits
test_predictions = torch.argmax(test_logits, dim=1)

# Convert predictions and labels to numpy arrays
test_predictions = test_predictions.cpu().numpy()
test_labels = test_labels_tensor.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

# Print the metrics
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")

#### Confusion Martix

In [ ]:
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Normalize the confusion matrix
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Set the style
sns.set(style="whitegrid")

# Plot the normalized confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cbar=True,
            xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])

plt.title('Adapted BERT Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

#### Receiver Operating Characteristic (ROC) Curve and Area Under Curve (AUC)

In [ ]:
# Move the test_logits tensor to the CPU
test_logits_cpu = test_logits.cpu().numpy()

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(test_labels, test_logits_cpu[:, 1])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Adapted BERT - ROC')
plt.legend(loc="lower right")
plt.show()